In [1]:
import pandas as pd

In [2]:
import oracledb

# --- Database Connection Details (Oracle XE) ---
dsn_tns = "localhost:1521/XEPDB1"
db_user = "sys"
db_password = "ordbpw"

# Attempt to connect to the database
connection = oracledb.connect(user=db_user, password=db_password, dsn=dsn_tns, mode=oracledb.SYSDBA)
cursor = connection.cursor()
print("Successfully connected to Oracle Database.")

Successfully connected to Oracle Database.


In [3]:
# --- 1. Create Database (Oracle XE uses instances/schemas, not separate databases like MySQL) ---
print(f"Using schema: {db_user}")

Using schema: sys


In [4]:
# --- 2. Define Schema (Create Tables) ---

# Create Banks Table
print("Creating BANKS table...")
cursor.execute("""
    BEGIN
        EXECUTE IMMEDIATE 'DROP TABLE REVIEWS';
    EXCEPTION
        WHEN OTHERS THEN
            IF SQLCODE != -942 THEN RAISE; END IF;
    END;
""") # Drop if exists
cursor.execute("""
    BEGIN
        EXECUTE IMMEDIATE 'DROP TABLE BANKS';
    EXCEPTION
        WHEN OTHERS THEN
            IF SQLCODE != -942 THEN RAISE; END IF;
    END;
""") # Drop if exists
cursor.execute("""
    CREATE TABLE BANKS (
        bank_id NUMBER GENERATED BY DEFAULT ON NULL AS IDENTITY PRIMARY KEY,
        bank_name VARCHAR2(255) UNIQUE NOT NULL
    )
""")
print("BANKS table created.")

# Create Reviews Table
print("Creating REVIEWS table...")
cursor.execute("""
    CREATE TABLE REVIEWS (
        review_id NUMBER GENERATED BY DEFAULT ON NULL AS IDENTITY PRIMARY KEY,
        bank_id NUMBER REFERENCES BANKS(bank_id),
        review_text CLOB, -- Use CLOB for potentially long review text
        sentiment_label VARCHAR2(50),
        sentiment_score NUMBER(5,4),
        identified_themes VARCHAR2(4000) -- Store themes as a delimited string or use a separate junction table if complex querying on themes is needed
    )
""")
print("REVIEWS table created.")
connection.commit()
print("Tables created successfully.")

Creating BANKS table...
BANKS table created.
Creating REVIEWS table...
REVIEWS table created.
Tables created successfully.


In [5]:
processed_df = pd.read_csv('../data/processed/processed_bank_app_reviews_with_sentiment.csv')
final_df = pd.read_csv('../data/processed/review_topics_and_sentiment.csv')

In [ ]:
# --- 3. Insert Cleaned Review Data using Python ---

print("\nInserting data into BANKS and REVIEWS tables...")

# Insert unique bank names into BANKS table
bank_names = processed_df['bank'].unique()
bank_id_map = {}

for bank_name in bank_names:
    try:
        cursor.execute("""
            INSERT INTO BANKS (bank_name) VALUES (:bank_name)
        """, bank_name=bank_name)
        # Retrieve the generated bank_id
        cursor.execute("SELECT bank_id FROM BANKS WHERE bank_name = :1", (bank_name,))
        bank_id = cursor.fetchone()[0]
        bank_id_map[bank_name] = bank_id
    except oracledb.IntegrityError:
        # If the bank already exists, retrieve its ID
        cursor.execute("SELECT bank_id FROM BANKS WHERE bank_name = :1", (bank_name,))
        bank_id = cursor.fetchone()[0]
        bank_id_map[bank_name] = bank_id

# Commit the transaction to save changes
connection.commit()

# Insert reviews into REVIEWS table
for index, row in processed_df.iterrows():
    cursor.execute("""
        INSERT INTO REVIEWS (bank_id, review_text, sentiment_label, sentiment_score, identified_themes)
        VALUES (:bank_id, :review_text, :sentiment_label, :sentiment_score, :identified_themes)
    """, bank_id=bank_id_map[row['bank']],
        review_text=row['review'],
        sentiment_label=row['distilbert_label'],
        sentiment_score=row['distilbert_score'],
        identified_themes=""
    )

# Commit the transaction to save changes
connection.commit()

# Insert identified themes from final_df into REVIEWS table
for index, row in final_df.iterrows():
    cursor.execute("""
        UPDATE REVIEWS
        SET identified_themes = :identified_themes
        WHERE review_id = :review_id
    """, identified_themes=row['identified_theme(s)'], review_id=row['review_id'])

# Commit the transaction to save changes
connection.commit()

print("\nData insertion complete.")


Inserting data into BANKS and REVIEWS tables...

Data insertion complete.


In [7]:
# --- Verify Insertion (KPI 2: Table populated with >1,000 entries) ---
cursor.execute("SELECT COUNT(*) FROM REVIEWS")
review_count = cursor.fetchone()[0]
print(f"\nTotal reviews in REVIEWS table: {review_count}")


Total reviews in REVIEWS table: 1200


In [8]:
# As a simplified demonstration of generating *some* SQL output from the data:
cursor.execute("""
    SELECT *
    FROM REVIEWS
""")
sample_reviews = cursor.fetchall()

In [9]:
columns = [col[0] for col in cursor.description]
print("\nSample reviews from REVIEWS table:")
for review in sample_reviews[:5]:  # Display first 5 reviews
    print(dict(zip(columns, review)))


Sample reviews from REVIEWS table:
{'REVIEW_ID': 1, 'BANK_ID': 1, 'REVIEW_TEXT': <oracledb.LOB object at 0x000001E170A3BB60>, 'SENTIMENT_LABEL': 'POSITIVE', 'SENTIMENT_SCORE': 0.9916, 'IDENTIFIED_THEMES': "['General']"}
{'REVIEW_ID': 2, 'BANK_ID': 1, 'REVIEW_TEXT': <oracledb.LOB object at 0x000001E170A385C0>, 'SENTIMENT_LABEL': 'POSITIVE', 'SENTIMENT_SCORE': 0.9993, 'IDENTIFIED_THEMES': "['User Interface/Experience (UI/UX)']"}
{'REVIEW_ID': 3, 'BANK_ID': 1, 'REVIEW_TEXT': <oracledb.LOB object at 0x000001E100CC48F0>, 'SENTIMENT_LABEL': 'POSITIVE', 'SENTIMENT_SCORE': 0.9972, 'IDENTIFIED_THEMES': "['General']"}
{'REVIEW_ID': 4, 'BANK_ID': 1, 'REVIEW_TEXT': <oracledb.LOB object at 0x000001E100CC6960>, 'SENTIMENT_LABEL': 'POSITIVE', 'SENTIMENT_SCORE': 0.9989, 'IDENTIFIED_THEMES': "['User Interface/Experience (UI/UX)']"}
{'REVIEW_ID': 5, 'BANK_ID': 1, 'REVIEW_TEXT': <oracledb.LOB object at 0x000001E100CC4770>, 'SENTIMENT_LABEL': 'NEGATIVE', 'SENTIMENT_SCORE': 0.9997, 'IDENTIFIED_THEMES': "[